In [ ]:
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
import torchvision.transforms as transforms
import torchvision.models as models
import pandas as pd
from tqdm import tqdm
from google.colab import drive

drive.mount('/content/drive')

data = np.load('/content/drive/MyDrive/Colab Notebooks/emnist-byclass-train.npz')
train_labels = data['training_labels']
train_images = data['training_images']

test_data = np.load('/content/drive/MyDrive/Colab Notebooks/emnist-byclass-test.npz')['testing_images']

train_images = torch.tensor(train_images, dtype=torch.float32).unsqueeze(1) / 255.0
train_labels = torch.tensor(train_labels, dtype=torch.long)
test_images = torch.tensor(test_data, dtype=torch.float32).unsqueeze(1) / 255.0

batch_size = 64
train_dataset = TensorDataset(train_images, train_labels)
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)

model = models.resnet18(pretrained=False)
model.conv1 = nn.Conv2d(1, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
model.fc = nn.Linear(model.fc.in_features, 62)
model = model.to('cuda' if torch.cuda.is_available() else 'cpu')

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

num_epochs = 20
device = 'cuda' if torch.cuda.is_available() else 'cpu'

for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    for images, labels in tqdm(train_loader):
        images, labels = images.to(device), labels.to(device)

        optimizer.zero_grad()

        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()

    print(f'Epoch [{epoch + 1}/{num_epochs}], Loss: {running_loss / len(train_loader):.4f}')

    torch.save(model.state_dict(), f'/content/drive/MyDrive/Colab Notebooks/model_epoch_{epoch + 1}.pth')

    model.eval()
    test_loader = DataLoader(test_images, batch_size=batch_size, shuffle=False)
    predictions = []

    with torch.no_grad():
        for test_batch in test_loader:
            test_batch = test_batch.to(device)
            outputs = model(test_batch)
            _, predicted = torch.max(outputs, 1)
            predictions.extend(predicted.cpu().numpy())

    df = pd.DataFrame({'Id': list(range(len(predictions))), 'Category': predictions})
    df.to_csv(f'/content/drive/MyDrive/Colab Notebooks/predictions_epoch_{epoch + 1}.csv', index=False)

print("Training completed and predictions saved.")


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


100%|██████████| 10906/10906 [04:18<00:00, 42.12it/s]


Epoch [1/20], Loss: 0.4631


100%|██████████| 10906/10906 [04:18<00:00, 42.24it/s]


Epoch [2/20], Loss: 0.3722


...

Epoch [18/20], Loss: 0.2444


100%|██████████| 10906/10906 [04:23<00:00, 41.41it/s]


Epoch [19/20], Loss: 0.2385


100%|██████████| 10906/10906 [04:23<00:00, 41.38it/s]


Epoch [20/20], Loss: 0.2322
Training completed and predictions saved.
